In [29]:
from datetime import datetime

In [30]:
class MyTradingStrategy:
    def __init__(self, name):
        self.__name = name

    def generate_signal(self, price_data):
        print("This methode is intended to be over ridden")
        return "Hold"

    @property
    def name(self):
        return self.__name 

In [31]:
class MySmaTradingStrategy(MyTradingStrategy):
    def __init__(self, swindow, lwindow):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__("MySmaTradingStrategy")

    def generate_signal(self, price_data):
        if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return "Hold"
        short_avg = sum(price_data[-self.__swindow:])/self.__swindow
        long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow

        if short_avg > long_avg:
            return "Buy"
        elif short_avg < long_avg:
            return "Sell"
        else:
            return "Hold"

    @property
    def swindow(self):
        return self.__swindow
    
    @property
    def lwindow(self):
        return self.__lwindow

In [32]:
class MyTrade:
    def __init__(self, strategy_name, signal, amount):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()

    def execute(self):
        print(f"Executing {self.__signal} trade with the strategy {self.__strategy_name} with amount {self.__amount} at the time {self.__timestamp}")

    @property
    def strategy_name(self):
        return self.__strategy_name
    
    @property
    def signal(self):
        return self.__signal
    
    @property
    def amount(self):
        return self.__amount
    
    @property
    def timestamp(self):
        return self.__timestamp

In [34]:
ObjStrategy = MySmaTradingStrategy(3, 5)
# ObjStrategy.generate_signal([1,2,3,4,5,6,7,8,9,10])
# print(ObjStrategy.lwindow)
# print(ObjStrategy.swindow)
# print(ObjStrategy.name)

strategy_name = ObjStrategy.name
signal = ObjStrategy.generate_signal([1,2,3,4,5,6,7,8,9,10])
ObjTrade = MyTrade(strategy_name, signal, 10000 )